In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("../")
os.getcwd()

'/home/mwirth/projects/sound-seperation'

In [3]:
from hydra import compose, initialize
from hydra.utils import instantiate
from tqdm.notebook import tqdm

In [11]:
with initialize(version_base="1.3", config_path="../configs/"):
    cfg = compose(config_name="train.yaml", return_hydra_config=True, #overrides=["experiment=HSN/HSN_Wav2vec2.yaml"]
                  )
cfg.datamodule.hf_dataset.cache_dir = "/home/mwirth/projects/sound-seperation/data/XCM"
cfg.datamodule.loaders.train.batch_size = 5

In [12]:
dm = instantiate(cfg["datamodule"], _recursive_=False)

In [13]:
dm.prepare_data();

Map:   0%|          | 0/80012 [00:00<?, ? examples/s]

Filter:   0%|          | 0/57582 [00:00<?, ? examples/s]

In [14]:
dm.setup("fit")
dm.train_set

Filter:   0%|          | 0/30744 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3843 [00:00<?, ? examples/s]

In [15]:
dm.train_set.hf_ds

Dataset({
    features: ['audio', 'filepath', 'start_time', 'end_time', 'ebird_code', 'length', 'quality', 'ebird_code_secondary', 'is_supervised'],
    num_rows: 30744
})

In [8]:
dl = dm.train_dataloader()

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [9]:
for data in tqdm(dl):
    break

NameError: name 'dl' is not defined

In [21]:
data["mix"].shape

torch.Size([5, 320000])

In [9]:
loss = instantiate(cfg.module.loss)

In [10]:
import torch

In [11]:
mixtures = torch.randn(10, 2, 32000)

In [12]:
est_sources = torch.randn(10, 6, 32000)

In [15]:
loss, reordered = loss(est_sources, mixtures, return_est=True)

In [30]:
module = instantiate(cfg.module, _recursive_=False)

/home/mwirth/miniconda3/envs/sound-separation/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `SignalNoiseRatio` from `torchmetrics` was deprecated and will be removed in 2.0. Import `SignalNoiseRatio` from `torchmetrics.audio` instead.
  _future_warning(


In [31]:
module

BaseModule(
  (model): TDConvNetpp(
    (bottleneck): Sequential(
      (0): FeatsGlobLN()
      (1): Conv1d(1, 128, kernel_size=(1,), stride=(1,))
    )
    (TCN): ModuleList(
      (0): Conv1DBlock(
        (shared_block): Sequential(
          (0): Conv1d(128, 512, kernel_size=(1,), stride=(1,))
          (1): PReLU(num_parameters=1)
          (2): FeatsGlobLN()
          (3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,), groups=512)
          (4): PReLU(num_parameters=1)
          (5): FeatsGlobLN()
        )
        (res_conv): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
        (skip_conv): Conv1d(512, 128, kernel_size=(1,), stride=(1,))
      )
      (1): Conv1DBlock(
        (shared_block): Sequential(
          (0): Conv1d(128, 512, kernel_size=(1,), stride=(1,))
          (1): PReLU(num_parameters=1)
          (2): FeatsGlobLN()
          (3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), groups=512)
          (4): PReLU(num_

In [ ]:
module = instantiate(cfg.module)

In [ ]:
data["mix"].std(axis=1)

In [ ]:
data["mix"].mean(axis=1)

In [11]:
from IPython.display import Audio

In [23]:
Audio(data["audio"]["wave"][0,0].numpy(), rate=32000)

In [24]:
Audio(data["audio"]["wave"][0,1].numpy(), rate=32000)

In [25]:
Audio(data["mix"][0], rate=32000)